<h1><center>Tarea 1: Roja o Azul</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Ortega 
- Nombre de alumno 2: María José Zambrano


### **Link de repositorio de GitHub:** [https://github.com/mzambrano1/Lab-programacion-cientifica](https://github.com/mzambrano1/Lab-programacion-cientifica)






# Temas a tratar

- Programación orientada a objetos.
- Manejo de datos en arreglos/tensores con `Numpy`.
- Análisis exploratorio de datos con `Pandas`
- Preprocesamiento con `Scikit-learn`.
- Composición de preprocesadores con Pipelines.

## Reglas:

- Fecha de entrega: Descrita en U-cursos
- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquier material del curso que estimen conveniente.



# Descripción del tarea (LEER ESTO).

En el presente notebook se presentan dos problemas, uno de **Segmentación de clientes en tiendas de retail** y otro enfocado en el **Análisis de temperaturas**. Producto de la larga extensión que podría significar la realización de los dos problemas, para efectos de esta tarea deben escoger **SOLAMENTE UNA** sección para desarrollar. Si desea puede borrar la pregunta que no realizará.

<p align="center">
  <img src="https://media1.giphy.com/media/ikwWGan6JZMo7bbf5L/giphy.gif?cid=790b761140b82e1c3d6754790ef49d96e8406bdac2f36d6f&rid=giphy.gif&ct=g">
</p>




#Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px

# Librerias utiles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder


     |████████████████████████████████| 13.2MB 272kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


----

# 1 Segmentación de Clientes en Tienda de Retail 🛍️

In [ ]:
HTML('<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/MW4d8_LLYp8?rel=0&amp;controls=0&amp;showinfo=0" frameborder="1" allowfullscreen></iframe></p>')

## 1.1 Cargar Dataset [Sin puntos]

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>


*Mr. Lepin Mora* quien es el gerente de una cotizada tienda de retail en Europa, les solicita si pueden analizar los datos de algunas de sus tiendas y si es posible extraer los diferenciar los tipos de clientes que posee el retail. 

Para esto, el área de ventas les entrega el archivo ```online_retail_II.xlsx``` con el que se les pide que cargue y visualicen algunas de las filas que componen el Dataset.

De la primera visualización de los datos, señale los atributos que componen el dataset.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#########################
df_2009 = pd.read_excel('/content/drive/MyDrive/Tarea1_lab/online_retail_II.xlsx',sheet_name=0)
df_2011 = pd.read_excel('/content/drive/MyDrive/Tarea1_lab/online_retail_II.xlsx',sheet_name=1)

#########################

In [8]:
df = df_2009.append(df_2011,ignore_index = True)

In [14]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
for col in df.columns:
  print(col)

Invoice
StockCode
Description
Quantity
InvoiceDate
Price
Customer ID
Country


## 1.2 Análisis Exploratorio de Datos [1 Punto]

En base a la primera visualización del dataset, *Don Mora* le solicita que realicen un análisis exploratorio de los datos, para esto les da **total libertad** para la revisión. De la revisión, ustedes deben explicar potenciales anomalías visualizadas y señalar si existe la necesidad de realizar una limpieza de datos.


In [20]:
df.shape

(1067371, 8)

In [45]:
df.nunique()

Invoice        53628
StockCode       5305
Description     5698
Quantity        1057
InvoiceDate    47635
Price           2807
Customer ID     5942
Country           43
dtype: int64

In [16]:
df.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


In [44]:
df['Country'].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland', 'Saudi Arabia',
       'Czech Republic', 'European Community'], dtype=object)

In [29]:
df.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [39]:
df[df.isna().any(axis=1)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1066997,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066998,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
1066999,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
1067000,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [41]:
descrptions_nan = df[df['Description'].isnull()]

In [46]:
descrptions_nan 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1060783,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
1060787,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
1060793,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [42]:
descrptions_nan[descrptions_nan['Customer ID'].isnull()]



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.0,NaN,United Kingdom
3114,489655,20683,NaN,-44,2009-12-01 17:26:00,0.0,NaN,United Kingdom
3161,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
3731,489781,84292,NaN,17,2009-12-02 11:45:00,0.0,NaN,United Kingdom
4296,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
1060783,581199,84581,NaN,-2,2011-12-07 18:26:00,0.0,NaN,United Kingdom
1060787,581203,23406,NaN,15,2011-12-07 18:31:00,0.0,NaN,United Kingdom
1060793,581209,21620,NaN,6,2011-12-07 18:35:00,0.0,NaN,United Kingdom
1062442,581234,72817,NaN,27,2011-12-08 10:33:00,0.0,NaN,United Kingdom


In [19]:
df_group_invoice = df.groupby('Invoice')

Quantity                           ... Customer ID                  
           count          mean        std  ...         50%      75%      max
Invoice                                    ...                              
489434       8.0     20.750000  12.646287  ...     13085.0  13085.0  13085.0
489435       4.0     15.000000   6.000000  ...     13085.0  13085.0  13085.0
489436      19.0     10.157895   5.899747  ...     13078.0  13078.0  13078.0
489437      23.0      6.304348   4.149861  ...     15362.0  15362.0  15362.0
489438      17.0     48.588235  14.013649  ...     18102.0  18102.0  18102.0
...          ...           ...        ...  ...         ...      ...      ...
C581484      1.0 -80995.000000        NaN  ...     16446.0  16446.0  16446.0
C581490      2.0    -11.500000   0.707107  ...     14397.0  14397.0  14397.0
C581499      1.0     -1.000000        NaN  ...     15498.0  15498.0  15498.0
C581568      1.0     -5.000000        NaN  ...     15311.0  15311.0  15311.0
C581569      2.0     -3.000000   2.828427  ...     17315.0  17315.0  17315.0

[53628 rows x 24 columns]

In [22]:
df_group_client = df.groupby('Customer ID')

In [24]:
df_group_client.describe()

Quantity                           ... Price                
               count       mean           std  ...   50%     75%     max
Customer ID                                    ...                      
12346.0         48.0   1.083333  15309.385753  ...  7.49  7.4900  142.31
12347.0        253.0  12.988142     16.477614  ...  1.95  3.7500   12.75
12348.0         51.0  53.215686     48.700848  ...  0.55  1.2500   40.00
12349.0        180.0   8.994444      7.846643  ...  2.55  6.2750  300.00
12350.0         17.0  11.588235      4.345383  ...  1.65  2.1000   40.00
...              ...        ...           ...  ...   ...     ...     ...
18283.0        986.0   1.757606      1.525774  ...  1.65  2.0800   15.95
18284.0         29.0  17.000000     14.992855  ...  1.25  2.5500   50.00
18285.0         12.0  12.083333     21.760926  ...  3.75  8.0875   50.00
18286.0         70.0   8.457143      7.224527  ...  2.10  5.8250   50.00
18287.0        156.0  19.301282     14.882516  ...  1.65  2.5500   50.00

[5942 rows x 16 columns]

**Justifique:**

Luego de realizar la exploración inicial se encontraron algunas anomalías. Primero se tienen cantidades de negativas de algunos objetos comprados al igual que se tienen precios negativos, cosas que resultan extrañas considerando que son tiendas que venden productos. También se encontraron valores NaN en las descripciones y en los id de los clientes. Se comprobó que todas las filas con NaN en la descripción, también tienen el id del cliente en NaN.

## 1.3 Primeras Visualizaciones [0,5 Puntos]

### 1.3.1 Obtención de TOPs 

Sin considerar los comentarios realizados en la sección 1.2, se les pide obtener el **Top de 30 productos más vendidos** y el **Top de 30 productos que generan más ganancias** para la tienda de retail. Deben considerar todo el registro temporal presente en el dataset y entregar la información en un gráfico de barras (Utilice `plotly`). ¿Los artículos más vendidos son los mismos que generan más ganancias?, Comente.

In [124]:
df['ganancia'] = df['Quantity']*df['Price']

In [125]:
df_product_grouped = df.groupby('StockCode')

In [126]:
sum = df_product_grouped.sum().reset_index()
sum['StockCode'] = sum['StockCode'].astype(str)
sum = sum.sort_values('Quantity')

In [127]:
fig = px.bar(sum, x='StockCode', y='Quantity')
fig.show()

In [128]:
best_30 = sum.tail(30)

In [129]:
fig = px.bar(best_30 , x='StockCode', y='Quantity')
fig.show()

In [130]:
sum = sum.sort_values('ganancia')

In [131]:
fig = px.bar(sum, x='StockCode', y='ganancia')
fig.show()

In [133]:
ganacia_best_30 = sum.tail(30)

In [134]:
fig = px.bar(ganacia_best_30, x='StockCode', y='ganancia')
fig.show()

In [139]:
best_30.merge(ganacia_best_30,on='StockCode')['StockCode']

0    85099F
1     22086
2     22386
3     20725
4     22197
5     84879
6    85099B
7    85123A
Name: StockCode, dtype: object

**Justifique:**

Se repiten solo 7 de los 30 productos de cada clasificación. Lo anterior puede explicarse bajo la hipótesis que, en primer lugar, no se considera el precio de costo de los artículos, por lo que los artículos más caros estarían favorecidos en el ranking de ganacias, mientras que los artículos más vendidos usualmente poseen un precio bajo o moderado.

### 1.3.2 Visualización del registro temporal 

El dueño del retail en su afán por saber más sobre los datos de su firma les solicita que grafiquen las ventas respecto al tiempo. Con esto les aclara que durante el día tienen muchas variaciones en sus ventas, por lo que les recomienda que consideren el registro temporal como `año-mes-día`. ¿Es posible observar datos extraños?, Comente lo que observa del gráfico.

**Hint**: Para realizar esta task generen una función (quizás en el futuro les sea necesario 😮).

In [151]:
df['fecha'] = pd.to_datetime(df['InvoiceDate'],format = '%Y%m%d').dt.date

In [153]:
grouped_date = df.groupby('fecha')

In [156]:
suma_fecha = grouped_date.sum().reset_index()

In [160]:
fig = px.line(suma_fecha, x="fecha", y="Quantity")
fig.show()

In [158]:
fig = px.line(suma_fecha, x="fecha", y="ganancia")
fig.show()

**Justifique:**

En los gráficos se úede ver que hay datos extraños, se observan ganacias y cantidades de productos vendidos negativos, como también peaks muy altos (que se alejan mucho de la tendencia central).

## 1.4 Limpieza de Datos [1,5 Puntos]
Como pudo ver en 1.1, 1.2 y 1.3 los datos presentan valores "extraños", es por esto que se le solicita que realice una **función** que permita **limpiar el dataset**. Realice esta función en base observaciones propias y justifique su elección.

Una vez realizada la función, **grafique nuevamente** el registro de ventas en el tiempo. ¿Es posible ver cambios?, Comente que observa.

In [ ]:
#########################
    Código Aquí
#########################

**Justificación de la limpieza:**

```
Escriba aquí su justificación
```

**Comentarios sobre el gráfico:**

```
Escriba aquí su justificación
```

### 1.4.1 Gráfico

**Justifique:**

```
Escriba aquí su justificación
```

## 1.5 Creación de nuevas Caracteristicas [1.5 Puntos] 

El area de ventas le comenta que existe una metodologia especial para realizar la segmentación de clientes, esta recive como nombre **LRMFP** y cada una de sus siglas se define como:

- Length (L): Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- Recency (R): Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, t_{m-i+1}; t_{m} es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- Monetary (M): El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- Frequency (F): Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- Periodicity (P): Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

Diseñe una función que permita obtener las caracteristicas **LRMFP**. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto. 

Una referencia interesante es [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.


In [ ]:
#########################
    Código Aquí
#########################

**Ejemplo de resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

## 1.5.1 Visualización de información y Outliers [0,5]

Visualice las diferenes caracteristicas obtenidas y comente que observa de ellas. ¿Es posible observar valores anómalos?. Proponga un mecanísmo sencillo para solucionar este problema (NO LO IMPLEMENTE).

**Justifique:**

```
Escriba aquí su justificación
```

### 1.5.1.1 Implementación del filtro de outliers [Bonus - 0,5 puntos]

Implemente el filtro de outliers como un [Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) de `Scikit-learn`.

In [ ]:
#########################
    Código Aquí
#########################

## 1.6 Segmentación [1 Punto]



Finalmente *Don Mora* le pregunta si seria posible realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de Clusters para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.6.1 Estandarizar Caracteristicas

Construya una función que nos permita estandarizar los atributos generados anteriormente. Para esto recuerde que una estandarización basica es:

$$Z = \dfrac{x-\mu}{s}$$

Con esto buscamos que cada atributo tenga un $\mu$ igual a 0 y una $\sigma$ igual a 1 (compruebe esto).

In [ ]:
#########################
    Código Aquí
#########################

**Comprobación:**

In [ ]:
print("Promedio: \n",ccustomers.mean(),"\n")
print("Desviación estandar: \n",ccustomers.std())

Promedio: 
 Length         6.770260e-17
Recency        2.733733e-17
Frequency     -1.794012e-17
Monetary       3.467400e-16
Periodicity   -8.144246e-17
dtype: float64 

Desviación estandar: 
 Length         1.0
Recency        1.0
Frequency      1.0
Monetary       1.0
Periodicity    1.0
dtype: float64


### 1.6.2 Clustering

Estandarizados los datos un compañero de trabajo les facilita un Código para evaluar la cantidad de cluster óptimos para separar los datos. (Otra forma seria revisando la Silhoutte).

En esta parte solo se le pedirá reemplazar la variable ```ccustomers``` por la del dataframe estandarizado.

In [ ]:
# Usamos metodo del codo para escoger el k que mejor ajusta el cluster.
elbow = pd.DataFrame(columns=['SSE','k'])
for k in range(1, 21):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(ccustomers)
    elbow = elbow.append({'SSE': kmeans.inertia_, 'k': k}, ignore_index=True)

# Son ploteados los Resultados
fig = go.Figure(data=[
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='lines'),
    go.Scatter(x = elbow.k, y=elbow.SSE, mode='markers')
])

fig.update_layout(title="Evaluación de los Clusters (Metodo del Codo):",
                 xaxis_title = "Numero de Clusters:",
                 yaxis_title = "SSE",
                 showlegend=False)
fig.show()

### 1.6.3 Segmentación de Clientes 🎁

En base al gráfico anterior escoja un valor que encuentren correcto. Para esto cambie el valor de "k" en el codigo de mas abajo.

In [ ]:
k=6
model = KMeans(n_clusters=k, random_state=80)
model.fit(ccustomers)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=80, tol=0.0001, verbose=0)

Genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique.


In [ ]:
customers["Cluster"] = model.labels_
#########################
    Código Aquí
#########################

**Justifique:**

```
Escriba aquí su justificación
```


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
|         | mean    | mean      | mean     | mean        | mean  | count |
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

--------------------------------------------------------------------------------

# 2 Clima Chileno ☀️

In [ ]:
HTML('<p align="center"><iframe width="560" height="315" src="https://www.youtube.com/embed/fGr-njYF_U0?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></p>')

## 2.1 Cargar Dataset

<p align="center">
  <img src="https://cdn.worldvectorlogo.com/logos/kiwi-3.svg" width='150px'>
</p>

`Kiguix`, Una prestigiosa empresa de software esta interesada en ingresar al mundo de la predicción del tiempo, por lo que se ha puesto en contacto con ustedes (unos expertos en Data Science) para analizar los datos.

Para comenzar el administrador de ```Kiguix``` les entrega el dataset llamado ```cr2_tasDaily_2018.csv``` el que posee un gran registro de temperaturas obtenidas en mas de **300 estaciones metereologicas*** de Chile. Con esto les solicita si pueden explorar este dataset y ver si es posible observar caracteristicas relevantes de los datos.

Como **primera Task** se le solicita cargar el archivo y mostrar una pequeña muestra del dataset a estudiar.

In [ ]:
#########################
    Código Aquí
#########################

## 2.2 Exploración y limpieza de datos

Cargado los datos, el administrador les consulta si es posible realizar un analisis de los datos en el estado que se encuentran. ¿Es posible observar información relevante al realizar un analisis exploratorio de los datos?. Recuerde justificar su respuesta en la casilla **Justifique**, utilice como argumento lo visto en la exploración de datos realizada.

### 2.2.1 ¿Es Posible Realizar en Este Estado un Análisis Exploratorio de los Datos? [0,5]


In [ ]:
#########################
    Código Aquí
#########################

**Justificación**

```
Escriba aqui su respuesta
```

### 2.2.2 Limpieza de datos [1 Punto]

Omitiendo su respuesta anterior, el jefe de ```Kiguix``` les comenta que está interesado en generar un Dataset "ordenado" para las estaciones meteorológicas que llevan por **Nombre**:

- 'Pichoy Valdivia Ad.'
- 'Pudahuel Santiago'
- 'Santo Domingo Ad.'
- 'Juan Fernandez Estacion Meteorologica'
- 'Futaleufu Ad.'
- 'Santa Juana'
- 'Lautaro Embalse'

Con esto les solicita que creen un DataFrame con columnas que contengan (1) el año en que fue tomada la muestra, (2) el mes en que se tomó la muestra, (3) la fecha del año y mes en que se tomó la muestra, (4) altura a nivel del mar que se encuentra la estación, (5) latitud y (6) longitud de la estación, (7) el nombre de la estación meteorológica y (8) la temperatura promedio del mes. Su jefe es crítico en señalar que necesita las columnas en este orden y que deja en usted el cargo de tomar las asunciones de que valores omitir en el DataFrame. En caso de ser necesario justifique asunciones.



**Ejemplo de tabla solicitada:**

|      |   Year | Month     | Dates               |   altura |   latitud |   longitud | station                               |      Temp |
|-----:|-------:|:----------|:--------------------|---------:|----------:|-----------:|:--------------------------------------|----------:|
|    0 |   1961 | August    | 1961-08-01 00:00:00 |      347 |  -43.1889 |   -71.8525 | Futaleufu Ad.                         |  6.2      |
|    1 |   1961 | December  | 1961-12-01 00:00:00 |      347 |  -43.1889 |   -71.8525 | Futaleufu Ad.                         | 13        |
|    2 |   1963 | April     | 1963-04-01 00:00:00 |       35 |  -33.6367 |   -78.835  | Juan Fernandez Estacion Meteorologica | 17.9      |
|    3 |   1965 | January   | 1965-01-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 18.0818   |
|    4 |   1965 | February  | 1965-02-01 00:00:00 |      347 |  -43.1889 |   -71.8525 | Futaleufu Ad.                         | 14.8      |
|    5 |   1965 | February  | 1965-02-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 21.8214   |
|    6 |   1965 | March     | 1965-03-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 20.7759   |
|    7 |   1965 | April     | 1965-04-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 18.244    |
|    8 |   1965 | May       | 1965-05-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 15.7233   |
|    9 |   1965 | June      | 1965-06-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 16.1467   |
|   10 |   1965 | July      | 1965-07-01 00:00:00 |      560 |  -28.6672 |   -70.6625 | Santa Juana                           | 14.4516   |

In [ ]:
#########################
    Código Aquí
#########################

**Justifique:**

```
 Escriba aqui su respuesta
```

### 2.2.3 Nueva exploración de datos [1 Punto]

Visto que el nuevo DataFrame entrega una estructura mas ordenada de los datos se decide hacer una nueva exploración de los datos, para esto se les pide que:

- Aplicar las 3 técnicas de EDA que usted estime conveniente.
- Realicen un Box-plot temporal de los Datos utilizando plotly.
- Graficar tendencia de la temperatura para cada uno de los meses del año.

Comente que puede observar de los datos. ¿Es posible observar alguna tendencia o patron?. Justifique.

In [ ]:
#########################
    Código Aquí
#########################

In [ ]:
#########################
    Código Aquí
#########################

**Justifique:**

```
 Escriba aqui su respuesta
```

### 2.2.3.1 Representación interesante [Bonus - 0,5 Puntos]


Implemente la siguiente visualización y proponga otra que considere útil.
Visite la referencia de [plotly](https://plotly.com/python) y [px.line](https://plotly.com/python/line-charts/) para mayor información.

In [ ]:
#########################
    Código Aquí
#########################

## 2.3 Pipeline y Predicción

### 2.3.1 Encoding [2 Puntos]

Contentos con su desempeño ```Kiguix``` les pide realizar su tarea más ambiciosa, la que consiste en crear un pipeline para predecir temperaturas futuras. Para esto les señala que desea realizar el siguiente encoding para las columnas del DataFrame creado en la sección ```2.2.2```:

- Codificar el atributo categórico ```station```.
- Obtener el MinMaxScaler de los atributos ```latitud```, ```longitud``` y ```altura```.
- Realizar una codificación cíclica de ```Year``` y ```Month``` ([referencia de ayuda](https://stats.stackexchange.com/questions/311494/best-practice-for-encoding-datetime-in-machine-learning)).

Para el caso de la codificación cíclica deberá crear una clase ```class CyclicalEncoding(BaseEstimator, TransformerMixin)``` para la codificación cíclica, por lo que se recomienda ver la [documentación](https://sklearn-template.readthedocs.io/en/latest/user_guide.html).




In [ ]:
#########################
    Código Aquí
#########################

### 2.3.2 Pipeline! [1 Punto]

Realizado el encoding añadan este a un pipeline, anexando como último paso un modelo de regresión. Para el modelo de regresión puede escoger uno de su preferencia o escoger un ```DecisionTreeRegressor()``` o ```LogisticRegression()```. 

Hecho esto, ```Kiguix``` en su rol de querer saber fehacientemente el desempeño de este modelo le sugiere utilizar R2 score, el cual es un estadístico que nos permite visualizar que tan bien predice un modelo de regresión. ¿Que tal le va a la regresión?, ¿Sera coherente utilizar solo estos atributos para predicciones?, Justifique su respuesta.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        dataset.drop(columns='Temp'),
        dataset.Temp,
        shuffle=True,
        test_size=0.3,
        random_state=42)

##############################
    Código Aquí Pipeline
##############################

pipeline.fit(X_train, y_train)
predicted_regression = pipeline.predict(X_test)

# Metrica util para medir regresiones
R2score = r2_score(y_test, predicted_regression)

**Justifique:**

```
 Escriba aqui su respuesta
```

### 2.3.3 Predicción del Futuro 🔮 [0,5 Puntos]

En su afán por el proyecto, el don *Dimadon Vallejos, CEO de `Kiguix`* les entrega el siguiente codigo para visualizar las predicciones. ¿Es coherente lo realizado?, comente.

In [ ]:
# Filtramos ciudad
data_santiago = dataframe_train[dataframe['station']=='Pudahuel Santiago']

# Creamos Información para predecir los proximos años
Prox_anho = data_santiago[data_santiago['Year']==2017][['Year', 'Month','Dates', 'altura', 'latitud',	'longitud', 'station']]
Prox_anho.Year.replace(2017,2018, inplace=True)

# Definimos el regresor entrenado anteriormente
rgr = regressor

# Predecimos
temp_2018 = rgr.predict(Prox_anho)

# Generamos el nuevo DataFrame
temp_2018 = {'Month':data_santiago['Month'].unique(), 'Temp':temp_2018}
temp_2018=pd.DataFrame(temp_2018)
temp_2018['Year'] = 2018
temp_2018

,Month,Temp,Year
0,1,17.654839,2018
1,2,17.879641,2018
2,3,14.263447,2018
3,4,11.925578,2018
4,5,9.131106,2018
5,6,7.234667,2018
6,7,9.131106,2018
7,8,11.925578,2018
8,9,14.263447,2018
9,11,17.654839,2018


In [ ]:
import plotly.graph_objects as go

forecasted_temp = pd.concat([data_santiago,temp_2018], sort=False).groupby(by='Year')['Temp'].mean().reset_index()
fig = go.Figure(data=[
    go.Scatter(x=forecasted_temp['Year'], y=forecasted_temp['Temp'], mode='lines'),
    go.Scatter(x=forecasted_temp ['Year'], y=forecasted_temp['Temp'], mode='markers')
])
fig.update_layout(title='Temperatura predicha en el tiempo:',
                 xaxis_title='Time', yaxis_title='Grados Celcius')
fig.show()

**Justifique:**

```
 Escriba aqui su respuesta
```

# Cierre
Recuerden que la tarea tiene un plazo de entrega de dos semanas y que **los días de atraso son válidos para las tareas**. Cualquier duda de la tarea, no duden en contactarnos por mail o U-cursos.



<p align="center">
  <img src="https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif">
</p>